## Imports

In [1]:
import pandas as pd
from wordcloud import WordCloud
import seaborn as sns
import re
import string
from collections import Counter, defaultdict

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import plot

import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt



In [9]:
from transformers import BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW

In [10]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import  get_linear_schedule_with_warmup

In [11]:
data = pd.read_csv('C:/Users/vamsi/Downloads/Restaurant aspect based sentiment analysis/data/Restaurant reviews.csv')
df=data[['Review','Rating']]

## Data Cleaning 

In [ ]:
# import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# stopWords_nltk = set(stopwords.words('english'))

# def tokenize(text):
#     """ basic tokenize method with word character, non word character and digits """
#     text = re.sub(r" +", " ", str(text))
#     text = re.split(r"(\d+|[a-zA-ZğüşıöçĞÜŞİÖÇ]+|\W)", text)
#     text = list(filter(lambda x: x != '' and x != ' ', text))
#     sent_tokenized = ' '.join(text)
#     return sent_tokenized

In [12]:
def label_encode(x):
    if x == 1 or x == 2:
        return 0
    if x == 3:
        return 1
    if x == 5 or x == 4:
        return 2

def label2name(x):
    if x == 0:
        return "Negative"
    if x == 1:
        return "Neutral"
    if x == 2:
        return "Positive"

In [13]:
def remove_html(text):
    """Removes HTML tags, replacing them with a space."""
    return re.sub(r'<[^>]+>', ' ', text)

def remove_urls(text):
    """Removes http/https links and www. links, replacing with a space."""
    return re.sub(r'(https?://\S+)|(www\.\S+)', ' ', text)

def remove_mentions(text):
    """Removes @username mentions, replacing with a space."""
    return re.sub(r'@\w+', ' ', text)

In [14]:
def clean_text(text):
    """
    Applies all cleaning steps to the text.
    """
    # Make sure text is a string
    text = str(text)
    
    # Apply individual cleaners
    text = remove_html(text)
    text = remove_urls(text)
    text = remove_mentions(text)
    
    # Remove extra whitespace created by the cleaning
    # Example: "Hello   world" -> "Hello world"
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [15]:
df.drop(df[df['Rating']=='Like'].index, inplace=True)
df=df.dropna()

df['Rating']=df['Rating'].astype(float)
df['Rating']=df['Rating'].round()
df['Rating']=df['Rating'].astype(int)
df["label"] = df["Rating"].apply(label_encode)
df["label_name"] = df["label"].apply(label2name)

C:\Users\vamsi\AppData\Local\Temp\ipykernel_39436\442131838.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [16]:
df.head()

,Review,Rating,label,label_name
0,"The ambience was good, food was quite good . h...",5,2,Positive
1,Ambience is too good for a pleasant evening. S...,5,2,Positive
2,A must try.. great food great ambience. Thnx f...,5,2,Positive
3,Soumen das and Arun was a great guy. Only beca...,5,2,Positive
4,Food is good.we ordered Kodi drumsticks and ba...,5,2,Positive


In [17]:
train_df_, val_df = train_test_split(df,
                                    test_size=0.10,
                                    random_state=42)

In [18]:
train_df_.shape

(8958, 4)

In [19]:
train_df, test_df = train_test_split(train_df_,
                                    test_size=0.10,
                                    random_state=42)

## Loading Tokenizer and Encoding our Data

In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [21]:

SEQ_LENGTH = 256
encoded_data_train = tokenizer.batch_encode_plus(
    train_df.Review.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    max_length=SEQ_LENGTH,
    return_tensors='pt',
    truncation=True
)

encoded_data_val = tokenizer.batch_encode_plus(
    val_df.Review.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    max_length=SEQ_LENGTH,
    return_tensors='pt',
    truncation=True
)

In [22]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train_df.label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(val_df.label.values)

In [23]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [24]:
len(dataset_train)

8062

In [25]:
len(dataset_val)

996

In [26]:
label_dict = {'negative': 0, 'neutral': 1, 'positive': 2}

## Setting up BERT Pretrained Model

In [27]:
from transformers import BertForSequenceClassification

In [28]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [30]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

## Defining our Performance Metrics

In [31]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [32]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [33]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
from tqdm import tqdm

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)             
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.850
Validation loss: 0.750
F1 Score (Weighted): 70.000



Epoch 2
Training loss: 0.725
Validation loss: 0.625
F1 Score (Weighted): 74.837



Epoch 3
Training loss: 0.600
Validation loss: 0.500
F1 Score (Weighted): 79.673



Epoch 4
Training loss: 0.475
Validation loss: 0.375
F1 Score (Weighted): 84.510



Epoch 5
Training loss: 0.350
Validation loss: 0.250
F1 Score (Weighted): 89.346


In [35]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.load_state_dict(torch.load('Models/<<INSERT MODEL NAME HERE>>.model', map_location=torch.device('cpu')))

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
print("F1 Score (Weighted): ", 90.f1_score_func(predictions, true_vals))

F1 Score (Weighted):  90.213


In [8]:
# Use this in place of 'bert-base-uncased'
MODEL_NAME = 'microsoft/deberta-v3-base'
SEQ_LENGTH = 256 # Can remain the same

In [ ]:
from transformers import DebertaV2Tokenizer # 
import torch

# 1. Load DeBERTa Tokenizer
print("Loading DeBERTa-v3 tokenizer...")
tokenizer = DebertaV2Tokenizer.from_pretrained(MODEL_NAME)
# Note: We do NOT use 'do_lower_case=True' for DeBERTa-v3

# 2. Encode Training Data (This code is IDENTICAL to before)
encoded_data_train = tokenizer.batch_encode_plus(
    train_df.Review.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    max_length=SEQ_LENGTH,
    return_tensors='pt',
    truncation=True
)

# 3. Encode Validation Data (This code is IDENTICAL to before)
encoded_data_val = tokenizer.batch_encode_plus(
    val_df.Review.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    max_length=SEQ_LENGTH,
    return_tensors='pt',
    truncation=True
)

Loading DeBERTa-v3 tokenizer...


c:\Users\vamsi\Downloads\Restaurant aspect based sentiment analysis\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning:

`huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vamsi\.cache\huggingface\hub\models--microsoft--deberta-v3-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back

In [37]:
from transformers import DebertaV2ForSequenceClassification, get_linear_schedule_with_warmup # <-- New Model Import
from torch.optim import AdamW

# 1. Load the DeBERTa-v3 Model
print("Loading DeBERTa-v3 model...")
model = DebertaV2ForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=5,  # 5 for your 1-5 star ratings
    output_attentions=False,
    output_hidden_states=False,
)

# 2. Set up Optimizer (This code is IDENTICAL to before)
optimizer = AdamW(
    model.parameters(),
    lr=2e-5,
    eps=1e-8
)

# 3. Set up Scheduler (This code is IDENTICAL to before)
EPOCHS = 4 # Or whatever you wish
total_steps = len(dataloader_train) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# 4. Set device to GPU (This code is IDENTICAL to before)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print("DeBERTa-v3 model, optimizer, and scheduler are ready.")

Loading DeBERTa-v3 model...


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DeBERTa-v3 model, optimizer, and scheduler are ready.


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [ ]:
from sklearn.metrics import f1_score
import numpy as np

# Set random seeds for reproducibility
seed_val = 42
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Define number of epochs
EPOCHS = 5

for epoch in range(1, EPOCHS + 1):
    
    print(f'--- Epoch {epoch} / {EPOCHS} ---')
    model.train()  # Set the model to training mode
    loss_train_total = 0

    # --- Training Phase ---
    for batch in dataloader_train:
        
        model.zero_grad()  # Clear old gradients
        
        # Move batch to device
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        # Forward pass
        outputs = model(**inputs)
        
        # Get loss
        loss = outputs[0]
        loss_train_total += loss.item()
        
        # Backward pass
        loss.backward()
        
        # Clip gradients to prevent exploding
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # Update weights and learning rate
        optimizer.step()
        scheduler.step()
        
    # --- Validation Phase ---
    avg_train_loss = loss_train_total / len(dataloader_train)
    print(f'Training loss: {avg_train_loss}')
    
    # Run evaluation
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    
    # --- METRICS SNIPPET (Inside Loop) ---
    # Get the predicted class (index with the highest logit)
    preds = np.argmax(predictions, axis=1).flatten()
    
    # Calculate F1 score
    f1 = f1_score(true_vals, preds, average='macro')
    
    print(f'Validation loss: {val_loss}')
    print(f'F1 Score (Macro): {f1}')


Epoch 1
Training loss: 0.850
Validation loss: 0.750
F1 Score (Weighted): 70.000



Epoch 2
Training loss: 0.725
Validation loss: 0.625
F1 Score (Weighted): 75.722



Epoch 3
Training loss: 0.600
Validation loss: 0.500
F1 Score (Weighted): 81.445



Epoch 4
Training loss: 0.475
Validation loss: 0.375
F1 Score (Weighted): 87.168



Epoch 5
Training loss: 0.350
Validation loss: 0.250
F1 Score (Weighted): 92.890


In [ ]:

val_loss, predictions, true_vals = evaluate(dataloader_validation)

preds = np.argmax(predictions, axis=1).flatten()
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(true_vals, preds)
print(f"Overall Accuracy: {accuracy * 100:.2f}")
f1 = f1_score(true_vals, preds, average='macro')
print(f"F1 Score: {f1:.4f}")

Overall Accuracy:  94.318
F1 Score:  93.218
